<h1>Import titles using Pickle</h1>

In [5]:
# Dependencies
import pickle

def ReadFile(file_name, title_list):
    
    # Open the files ('rb' is for read binary)
    file_object = open(file_name,'rb')
    
    # Load files into list using pickle
    in_list = pickle.load(file_object)
    
    # Close files
    file_object.close()
    
    # Add list just read in to existing list
    title_list.extend(in_list)
    
    return(title_list)

In [6]:
# Crate empty list for left sites
left_title_list = []

# Read left sites
left_title_list = ReadFile("data/atlantic_titles", left_title_list)
left_title_list = ReadFile("data/BBC_titles", left_title_list)
left_title_list = ReadFile("data/mjones_titles", left_title_list)
left_title_list = ReadFile("data/newrepublic_titles", left_title_list)
left_title_list = ReadFile("data/politico_titles", left_title_list)
left_title_list = ReadFile("data/slate_titles", left_title_list)
left_title_list = ReadFile("data/thedailybeast_titles", left_title_list)
left_title_list = ReadFile("data/theguardian_titles", left_title_list)
left_title_list = ReadFile("data/theintercept_titles", left_title_list)

#***MAKE SURE THIS NUMBER MATCHES THE NUMBER OF DOCUMENTS IN MONGODB***
print(len(left_title_list))

# Crate empty list for right sites
right_title_list = []

# Read right sites
right_title_list = ReadFile("data/americanconservative_titles", right_title_list)
right_title_list = ReadFile("data/breitbart_titles", right_title_list)
right_title_list = ReadFile("data/dailywire_titles", right_title_list)
right_title_list = ReadFile("data/economist_titles", right_title_list)
right_title_list = ReadFile("data/fiscaltimes_titles", right_title_list)
right_title_list = ReadFile("data/foxnews_titles", right_title_list)
right_title_list = ReadFile("data/nypost_titles", right_title_list)
right_title_list = ReadFile("data/reason_titles", right_title_list)
right_title_list = ReadFile("data/thehill_titles", right_title_list)
right_title_list = ReadFile("data/washtimes_titles", right_title_list)

#***MAKE SURE THIS NUMBER MATCHES THE NUMBER OF DOCUMENTS IN MONGODB***
print(len(right_title_list))

57112
105279


<h1>Combine Right & Left Dataframes</h1>

In [7]:
# Dependencies
import pandas as pd
import numpy as np

# Bias of 1 -> Left; Bias of 0 -> Right
df1 = pd.DataFrame({'title': np.array(left_title_list), 'bias': 1})
df2 = pd.DataFrame({'title': np.array(right_title_list), 'bias': 0})

print(df1.head())
print(df1.shape)
print()
print(df2.head())
print(df2.shape)

df_combined = pd.concat([df1, df2])
print()
print(df_combined.shape)

   bias                                              title
0     1  Conservative High Schoolers Want to ‘Own the L...
1     1  The Instagram Forums Where Teens Go to Debate ...
2     1  The Doomed Republican Attempt to Impeach Rod R...
3     1                          Facebook Is Probably Fine
4     1                  Secretary of a State of Confusion
(57112, 2)

   bias                                        title
0     0               Fruits Of The Quiet Revolution
1     0               Trust And Mistrust In Churches
2     0   A Democratic President From Trump Country?
3     0  ‘Arab NATO’: A Terrible Idea That Won’t Die
4     0                       TAC Fall Intern Wanted
(105279, 2)

(162391, 2)


<h1>Logistic Regression on Words in Title</h1>

In [95]:
# Dependencies
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

stop_words_list = ['english', None]
ngrams_list = [(1,2), (1,3), (2,3)]
col_list = []
for item in stop_words_list:
    for ngram in ngrams_list:
        col_list.append([item, ngram])

results_list = []

for item in col_list:

    # Create a matrix of word count using CountVectorizer
    # Count single words and word pairs (ngram_range = 1-2)
    vectorizer = CountVectorizer(stop_words=item[0], ngram_range=item[1])

    # Fit and transform data
    X = vectorizer.fit_transform(df_combined['title'])

    # Create training and test split
    X_train, X_test, y_train, y_test  = train_test_split(X, df_combined['bias'], random_state=41)

    # Create the model and fit the training data
    logreg = LogisticRegression()
    logreg.fit(X_train,y_train)

    #logreg.score(X_test,y_test)

    a = zip(logreg.coef_.tolist()[0],list(vectorizer.get_feature_names()))
    b = list(a)
    b.sort()
    
    results_list.append(b)

In [96]:
# Show top 20 words/phrases for right and left
for i in range(0,20):
    print(b[i],b[len(b)-i-1])

(-2.8670418502121082, 'islamic state') (4.746869180273421, 'mother jones')
(-2.7377216207281942, 'dem senator') (4.307260082443813, 'country profile')
(-2.4200407355194113, 'bozell graham') (3.101874051225947, 'the minute')
(-2.384968191535314, 'illegal immigrant') (2.8427003928711514, 'stephen colbert')
(-2.372359970152218, 'report trump') (2.688193593697761, 'seth meyers')
(-2.3469184350716707, 'gop senator') (2.495900608357429, 'meghan markle')
(-2.2050028779278397, 'dem lawmaker') (2.4836027575380406, 'intercepted podcast')
(-2.160547051497032, 'illegal aliens') (2.3165487252252515, 'inside the')
(-2.1292422048128623, 'dem rep') (2.2702031735942354, 'radio atlantic')
(-2.067727731028444, 'gop rep') (2.151885911653669, 'after dark')
(-2.063520016802268, 'watch cnn') (2.0913745973395876, 'why did')
(-2.0587244679423025, 'illegal alien') (2.0836165270138154, 'russia inquiry')
(-2.055302303753218, 'the latest') (1.9934458500964183, 'syria war')
(-2.021334234913922, 'anti israel') (1.97

In [112]:
for i, item in enumerate(col_list):
    print("RIGHT", item)
    top_list = []
    for mytuple in results_list[i][0:50]:
        for word in mytuple:
            if type(word) is str:
                top_list.append(word)
    print(top_list)
    print()
    
for i, item in enumerate(col_list):
    print("LEFT", item)
    top_list = []
    for x in range(len(results_list[i])-1, len(results_list[i])-51, -1):
        mytuple = results_list[i][x]
        for word in mytuple:
            if type(word) is str:
                top_list.append(word)
    print(top_list)
    print()

RIGHT ['english', (1, 2)]
['brickbat', 'delingpole', 'walsh', 'things need', 'politics week', 'dem', 'leftists', 'wapo', 'nolte', 'erickson', 'prageru', 'shapiro', 'amnesty', 'prager', 'limbaugh', 'business week', 'malkin', 'things know', 'pollak', 'virgil', 'leftist', 'msm', 'gop senator', 'wh', 'flashback', 'cartel', 'markets', 'exclusive', 'report', 'watch', 'ca', 'guess', 'german', 'abbas', 'pinkerton', 'illegal immigrant', 'libertarian', 'auction', 'illegals', 'judge roy', 'live coverage', 'ag', 'probing', 'poland', 'view table', 'lieu', 'tories', 'kassam', 'coulter', 'tancredo']

RIGHT ['english', (1, 3)]
['brickbat', 'delingpole', 'walsh', 'dem', 'politics week', 'leftists', 'erickson', 'amnesty', 'nolte', 'wapo', 'prageru', 'shapiro', 'limbaugh', 'prager', 'leftist', 'business week', 'malkin', 'gop senator', 'markets', 'exclusive', 'report', 'virgil', 'watch', 'pollak', 'things know', 'flashback', 'msm', 'wh', 'cartel', 'german', 'things need', 'guess', 'ca', 'libertarian', 'ab

In [103]:
print(results_list[0][-1])

(3.973954890231789, 'mother jones')


In [198]:
logreg.predict_proba(vectorizer.transform(['syria war']))

array([[ 0.22263179,  0.77736821]])

In [119]:
# Dependencies
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Create a matrix of word count using CountVectorizer
# Count single words and word pairs (ngram_range = 1-2)
vectorizer = CountVectorizer(stop_words='english', ngram_range=(2,3))

# Fit and transform data
X = vectorizer.fit_transform(df_combined['title'])

# Create training and test split
X_train, X_test, y_train, y_test  = train_test_split(X, df_combined['bias'], random_state=42, 
                                                    stratify=df_combined['bias'])

# Create the model and fit the training data
logreg = LogisticRegression()
logreg.fit(X_train,y_train)

#logreg.score(X_test,y_test)

a = zip(logreg.coef_.tolist()[0],list(vectorizer.get_feature_names()))
b = list(a)
b.sort()

In [177]:
print(len(vectorizer.get_feature_names()))

1429053


<h1>Output Model Results to JSON File</h1>

In [204]:
d = {}

for i in range(0,50):
    for item in b[i]:
        if type(item) is str:
            d[item] = logreg.predict_proba(vectorizer.transform([item]))[0][0]
            
d
            
import json

# Write list of dictionaries to JS file
with open ('model-data.json', 'w') as outfile:
    json.dump([d], outfile)

In [125]:
# Show top 20 words/phrases for right and left
for i in range(0,50):
    print(b[i],b[len(b)-i-1])

(-2.8709587981589006, 'dem senator') (4.6244598013699685, 'mother jones')
(-2.847533708445915, 'islamic state') (4.0753937998460055, 'country profile')
(-2.766185831324463, 'illegal immigrant') (3.2942659437524675, 'trump dark')
(-2.5817079659390574, 'bozell graham') (2.7803726601483687, 'seth meyers')
(-2.547365925271587, 'illegal aliens') (2.6959559454819844, 'intercepted podcast')
(-2.530145724307294, 'dem lawmaker') (2.689758023257183, 'stephen colbert')
(-2.4951857981155876, 'gop senator') (2.53966974589822, 'meghan markle')
(-2.4366823881125983, 'report trump') (2.4868988575557447, 'radio atlantic')
(-2.4216887461593735, 'illegal alien') (2.238659191271554, 'trevor noah')
(-2.3796768895371394, 'dem rep') (2.1111985148542383, '45 dark')
(-2.312465233749302, 'anti israel') (2.091504224759743, 'russia inquiry')
(-2.3115555943607413, 'police say') (2.0820501926497603, 'letter africa')
(-2.276756997136163, 'illegal immigrants') (1.9560269595909763, 'michael flynn')
(-2.220404427852135

In [135]:
logreg.predict_proba(vectorizer.transform(['syria war']))

array([[ 0.22263179,  0.77736821]])

In [187]:
right_words = ['2nd amendment', 'daca amnesty', 'open borders', 'illegal immigrant', 'islamic state']

right_scores = []

for phrase in right_words:
    score = logreg.predict_proba(vectorizer.transform([phrase]))[0][0] * 100
    right_scores.append(score)
    
left_words = ['undocumented immigrants', 'muslim ban', 'syria war', 'family separations', 'russia inquiry']

left_scores = []

for phrase in left_words:
    score = -logreg.predict_proba(vectorizer.transform([phrase]))[0][1] * 100
    left_scores.append(score)
    
print(right_scores)
print(left_scores)

[90.550753572625112, 93.301483246755623, 94.016395249112975, 96.443661108029289, 96.712372869410416]
[-75.854714214487345, -77.511498328533079, -77.736821015758963, -80.368374524988155, -82.59889590187592]


In [197]:
import plotly
import plotly.graph_objs as go
import numpy as np

left_text = []
for i in range(0,5):
    text = left_words[i] + ' ' + str(round(left_scores[i],1)) + '% '
    left_text.append(text)
    
right_text = []
for i in range(0,5):
    text = right_words[i] + ' ' + str(round(right_scores[i],1)) + '% '
    right_text.append(text)

trace1 = go.Bar(
    y = np.arange(1,6),
    x = left_scores,
    text = left_text,
    textposition = 'auto',
    textfont = dict(size = 16, color = 'white'),
    hoverinfo = 'none',
    name = 'Left',
    orientation = 'h',
    marker = dict(color = 'blue'),
    offset = [-0.5,-1,-1,-1,0]
)
trace2 = go.Bar(
    y = np.arange(1,6),
    x = right_scores,
    text = right_text,
    textposition = 'auto',
    textfont = dict(size = 16, color = 'white'),
    hoverinfo = 'none',
    name = 'Right',
    orientation = 'h',
    marker = dict(color = 'red')
)

data = [trace1, trace2]

layout = go.Layout(
    barmode='group',
    title = '<b>Model Phrase Probability</b>',
    titlefont = dict(
        size = 24
    ),
    yaxis = dict(
        showticklabels = False
    ),
    xaxis = dict(
        title = 'Probability that article has left/right bias',
    )
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename = 'joint-bar.html')

'file://C:\\Users\\spenc\\Documents\\pattern\\joint-bar.html'

In [189]:
data1 = [trace1]
data2 = [trace2]

layout = go.Layout(
    barmode='group',
    title = '<b>Left Bias Probability</b>',
    titlefont = dict(
        size = 24
    ),
    yaxis = dict(
        showticklabels = False
    ),
    xaxis = dict(
        title = 'Probability that article bias is left',
    )
)

fig = go.Figure(data=data1, layout=layout)
plotly.offline.plot(fig, filename = 'left-bar.html')

layout = go.Layout(
    barmode='group',
    title = '<b>Right Bias Probability</b>',
    titlefont = dict(
        size = 24
    ),
    yaxis = dict(
        showticklabels = False
    ),
    xaxis = dict(
        title = 'Probability that article bias is right',
    )
)

fig = go.Figure(data=data2, layout=layout)
plotly.offline.plot(fig, filename = 'right-bar.html')

'file://C:\\Users\\spenc\\Documents\\pattern\\right-bar.html'